In [10]:
import pandas as pd
import numpy as np

In [6]:
# Read in data from crew csv
crew = pd.read_csv('../data/raw/title.crew.tsv', sep='\t')
ratings = pd.read_csv('../data/raw/title.ratings.tsv', sep='\t')
titles = pd.read_csv('../data/raw/title.basics.tsv', sep='\t')


C:\Users\Joeyz\AppData\Local\Temp\ipykernel_8844\2674795029.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles = pd.read_csv('../data/raw/title.basics.tsv', sep='\t')


In [17]:
titles.replace('\\N', np.nan, inplace=True)
titles['startYear'] = titles['startYear'].astype(float)

In [20]:
titles.tail()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
9273127,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010.0,NaN,NaN,"Action,Drama,Family"
9273128,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010.0,NaN,NaN,"Action,Drama,Family"
9273129,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010.0,NaN,NaN,"Action,Drama,Family"
9273130,tt9916856,short,The Wind,The Wind,0,2015.0,NaN,27,Short
9273131,tt9916880,tvEpisode,Horrid Henry Knows It All,Horrid Henry Knows It All,0,2014.0,NaN,10,"Adventure,Animation,Comedy"


In [22]:
between_1950_2020_titles = titles[(titles['startYear']>=1950) & (titles['startYear']<=2020) & (titles['titleType'] != 'tvEpisode')]['tconst']

In [23]:
#Heuristic: Let's filter out movies with less than 10000 votes

high_votes = ratings[ratings['numVotes'] > 10000]

In [25]:
votes_and_titles = pd.merge(between_1950_2020_titles, high_votes, on='tconst')

In [ ]:
#Turn the directors column into a list
crew['directors'] = crew['directors'].str.split(',')

In [32]:
# explode directors column
directors = crew[['tconst', 'directors']].explode('directors')


In [34]:
# Merge the directors with the titles
directors_and_titles = pd.merge(directors, votes_and_titles, on='tconst')

In [85]:
from sklearn.preprocessing import normalize

In [94]:
#Create 3 columns with votes, normalized votes, and rankings
director_votes = directors_and_titles.groupby('directors')['numVotes'].sum()
normalized_director_votes = normalize(director_votes.values.reshape(1, -1))
ranked_director_votes = director_votes.rank(ascending=False)
final_director_rankings = pd.DataFrame({'director':director_votes.index, 'rank':ranked_director_votes.values, 'total_votes':director_votes.values, 'normalized_votes': normalized_director_votes[0]})
final_director_rankings.to_csv('../data/interim/director_rankings.csv', index=False)

Now Repeat with the Writers

In [95]:
crew['writers'] = crew['writers'].str.split(',')
writers = crew[['tconst', 'writers']].explode('writers')

In [96]:
writers_and_titles = pd.merge(writers, votes_and_titles, on='tconst')

In [97]:

writer_votes = writers_and_titles.groupby('writers')['numVotes'].sum()
normalized_writer_votes = normalize(writer_votes.values.reshape(1, -1))
ranked_writer_votes = writer_votes.rank(ascending=False)
final_writer_rankings = pd.DataFrame({'writer':writer_votes.index, 'rank':ranked_writer_votes.values, 'total_votes':writer_votes.values, 'normalized_votes': normalized_writer_votes[0]})
final_writer_rankings.to_csv('../data/interim/writer_rankings.csv', index=False)